In [2]:
from flask import Flask, render_template,request
import plotly
import plotly.graph_objs as go

import pandas as pd
import numpy as np
import json

import webbrowser
from threading import Timer
import logging

In [3]:
budget = pd.read_csv('data/clean_budget.csv')
satellite = pd.read_csv('data/clean_satellite.csv')

In [3]:
budget[budget.value == budget.value.max()]

,application,year,value,country,program_type,program_name,unit,currency
3756,Earth Observation,2017,794753.561148,Japan,Defence,National program,million,USD


In [25]:
app = Flask(__name__)

@app.route('/')
def index():
    feature = 'Budget'
    bar = create_plot(feature)
    table = create_table(feature)
    return render_template('index.html', table=table, plot=bar)


def create_table(feature):
    print(feature)
    if feature == 'Satellite':
        df = satellite
    else:
        df = budget
    table = go.Figure([go.Table(
        header=dict(
            values=df.columns,
            line_color='darkslategray',
            fill_color='royalblue',
            align=['left','center'],
            font=dict(color='white', size=14),
            height=40
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns],
            align = "left",
            line_color='darkslategray',
            fill_color='whitesmoke',
            font=dict(color='midnightblue', size=12))
        )
    ])
 
    table.update_layout(
        autosize=True,
        height=500,
    )
    tableJSON = json.dumps(table, cls=plotly.utils.PlotlyJSONEncoder)

    return tableJSON

def create_plot(feature):    
    if feature == 'Bar':

        data = [
            go.Bar(
                x=budget['country'], # assign x as the dataframe column 'x'
                y=budget['value']
            )
        ]
    else:
        
        data = [go.Scatter(
            x = budget['year'],
            y = budget['value'],
            mode = 'markers'
        )]

    graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)

    return graphJSON


@app.route('/plot', methods=['GET', 'POST'])
def change_features():

    feature = request.args['selected']

    if feature == 'Satellite' or feature == 'Budget':
        tableJSON= create_table(feature)
        
        return tableJSON
        
    if feature == 'Bar' or feature == 'Scatter':
        graphJSON= create_plot(feature)

        return graphJSON

def open_browser():
      webbrowser.open_new('http://127.0.0.1:2000/')

if __name__ == '__main__':
    Timer(1, open_browser).start();
    app.logger.setLevel(logging.DEBUG)
    app.run(port=2000)
    
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:2000/ (Press CTRL+C to quit)


Budget


127.0.0.1 - - [17/Jan/2021 16:19:31] "GET / HTTP/1.1" 200 -


Satellite


127.0.0.1 - - [17/Jan/2021 16:19:36] "GET /plot?selected=Satellite HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2021 16:19:42] "GET /plot?selected=Scatter HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2021 16:19:45] "GET /plot?selected=Bar HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2021 16:19:49] "GET /plot?selected=Scatter HTTP/1.1" 200 -
